# Pandas

The `numpy` module is excellent for numerical computations, but to handle missing data or arrays with mixed types takes more work. The `pandas` module is currently the most widely used tool for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular `pandas` features:

* A fast and efficient "DataFrame" object for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Intelligent data alignment and integrated handling of missing data;
* Intelligent label-based slicing, fancy indexing, and subsetting of large data sets;
* Aggregating or transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Time series-functionalities;
* Highly optimized for performance, with critical code paths written in Cython or C.


In [76]:
import pandas as pd
import numpy as np

## Series

Series are completely equivalent to 1D array but with axis labels and the possibility to store heterogeneous elements. Of paramount importance are the time-series, used to define time evolutions of a phenomenon. 


In [77]:
from string import ascii_lowercase as letters

# Creating a series, accessing indexes, values and values by their index 
xs = pd.Series(np.arange(10)*0.5, index=tuple(letters[:10]))
print (xs,'\n')
print (xs.index,'\n')
# Values of the Series are actually a numpy array
print (xs.values, type(xs.values),'\n')
print (xs['f'], xs.f, xs.h, '\n')
print (xs[['d', 'f', 'h']], '\n')

a    0.0
b    0.5
c    1.0
d    1.5
e    2.0
f    2.5
g    3.0
h    3.5
i    4.0
j    4.5
dtype: float64 

Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype='object') 

[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5] <class 'numpy.ndarray'> 

2.5 2.5 3.5 

d    1.5
f    2.5
h    3.5
dtype: float64 



In [78]:
# Extracting elements and operations: same as numpy array
print (xs[:3],'\n')
print (xs[7:], '\n')
print (xs[::3], '\n')
print (xs[xs>3], '\n')
print (np.exp(xs), '\n')
print (np.mean(xs), np.std(xs), '\n')

a    0.0
b    0.5
c    1.0
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a    0.0
d    1.5
g    3.0
j    4.5
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a     1.000000
b     1.648721
c     2.718282
d     4.481689
e     7.389056
f    12.182494
g    20.085537
h    33.115452
i    54.598150
j    90.017131
dtype: float64 

2.25 1.4361406616345072 



In [79]:
# Series can be created from python dictionary too.
# Not that the elements can be whatever!
d = {'b' : 1, 'a' : 'cat', 'c' : [2,3]}
pd.Series(d)



b         1
a       cat
c    [2, 3]
dtype: object

A key difference between Series and ndarray is that operations between Series automatically align the data based on label. Thus, you can write computations without giving consideration to whether the Series involved have the same labels.

In [80]:
s = pd.Series(np.random.randn(5), index=tuple(letters[:5]))
s[1:] + s[:-1]

a         NaN
b    0.848730
c   -0.916913
d    0.859782
e         NaN
dtype: float64

### Time series

Time series are very often used to profile the behaviour of a quantity as a function of time. Pandas as a special index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`

In [81]:
# to define a date, the datetime module is very useful
import datetime as dt
date = dt.date.today()
date = dt.datetime(2018,11,19,14,45,10,15)
print (date)

# otherwise, several notations are interpreted too
date = 'Nov 19 2018'
date = '19/11/2018 14:45:00'
print (date)

days = pd.date_range(date, periods=7, freq='D')
print (days)

seconds = pd.date_range(date, periods=3600, freq='s')
print (seconds)


2018-11-19 14:45:10.000015
19/11/2018 14:45:00
DatetimeIndex(['2018-11-19 14:45:00', '2018-11-20 14:45:00',
               '2018-11-21 14:45:00', '2018-11-22 14:45:00',
               '2018-11-23 14:45:00', '2018-11-24 14:45:00',
               '2018-11-25 14:45:00'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2018-11-19 14:45:00', '2018-11-19 14:45:01',
               '2018-11-19 14:45:02', '2018-11-19 14:45:03',
               '2018-11-19 14:45:04', '2018-11-19 14:45:05',
               '2018-11-19 14:45:06', '2018-11-19 14:45:07',
               '2018-11-19 14:45:08', '2018-11-19 14:45:09',
               ...
               '2018-11-19 15:44:50', '2018-11-19 15:44:51',
               '2018-11-19 15:44:52', '2018-11-19 15:44:53',
               '2018-11-19 15:44:54', '2018-11-19 15:44:55',
               '2018-11-19 15:44:56', '2018-11-19 15:44:57',
               '2018-11-19 15:44:58', '2018-11-19 15:44:59'],
              dtype='datetime64[ns]', length=3600, fr

To learn more about the frequency strings, please see this [link](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)


Timestamped data is the most basic type of time series data that associates values with points in time. For pandas objects it means using the points in time.

functions like `pd.to_datetime` can be used, for instance, when reading information as string from a dataset

In [82]:
tstamp = pd.Timestamp(dt.datetime(2018, 11, 19))

# internally it counts the nanoseconds from January 1st 19
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print(tstamp.value)

# when creating a timestamp the format can be explicitly passed
print (pd.to_datetime('2010/11/12', format='%Y/%m/%d'))
print (pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M'))


1542585600000000000
2010-11-12 00:00:00
2010-11-12 00:00:00


A standard series can be created and (range of) elements can be used as indexes

In [83]:
tseries = pd.Series(np.random.normal(10, 1, len(days)), index=days)
# Extracting elements
print (tseries[0:4], '\n')
print (tseries['2018-11-19':'2018-11-21'], '\n') # Note - includes end time


2018-11-19 14:45:00    10.666782
2018-11-20 14:45:00    11.302499
2018-11-21 14:45:00     9.596325
2018-11-22 14:45:00     9.934479
Freq: D, dtype: float64 

2018-11-19 14:45:00    10.666782
2018-11-20 14:45:00    11.302499
2018-11-21 14:45:00     9.596325
Freq: D, dtype: float64 



`pd.to_datetime` can also be used to create a `DatetimeIndex`:

In [84]:
pd.to_datetime([1, 2, 3], unit='D', origin=pd.Timestamp('1980-02-03'))

DatetimeIndex(['1980-02-04', '1980-02-05', '1980-02-06'], dtype='datetime64[ns]', freq=None)

## DataFrame

A pandas DataFrame is like a simple tabular spreadsheet. For future reference (or for people already familiar with R), a pandas DataFrame is very similar to the R DataFrame.

Each column in a DataFrame is a Series object.

The element can be whatever, missing data are dealt with too (as NaN)

### DataFrame creation

A DataFrame can be created implicitly, with, e.g., a DatatimeIndex object as index:

In [85]:
entries=10
dates=pd.date_range('19/11/2018 14:45:00',freq='h', periods=entries)
df = pd.DataFrame(np.random.randn(entries,4), index=dates, columns=['A','B','C','D'])
df


,A,B,C,D
2018-11-19 14:45:00,-0.244284,1.045144,0.147562,0.634902
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,-0.222559
2018-11-19 16:45:00,1.053318,-0.293532,-0.781415,0.810136
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,0.758565
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,0.382025
2018-11-19 19:45:00,0.362288,-0.286442,1.171918,-0.611602
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,0.380381
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,-1.695351
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,-0.858397
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,-0.463744


or by means of a dictionary:


In [86]:
df2 = pd.DataFrame(
    { 'A' : 1.,
      'B' : pd.Timestamp('20130102'),
      'C' : pd.Series(1,index=range(4),dtype='float32'),
      'D' : np.arange(7,11),
      'E' : pd.Categorical(["test","train","test","train"]),
    }
    )
   

In [87]:
df2

,A,B,C,D,E
0,1.0,2013-01-02,1.0,7,test
1,1.0,2013-01-02,1.0,8,train
2,1.0,2013-01-02,1.0,9,test
3,1.0,2013-01-02,1.0,10,train


### Viewing Data

In [88]:
df.head()

,A,B,C,D
2018-11-19 14:45:00,-0.244284,1.045144,0.147562,0.634902
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,-0.222559
2018-11-19 16:45:00,1.053318,-0.293532,-0.781415,0.810136
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,0.758565
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,0.382025


In [89]:
df.tail(4)

,A,B,C,D
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,0.380381
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,-1.695351
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,-0.858397
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,-0.463744


In [90]:
df.index

DatetimeIndex(['2018-11-19 14:45:00', '2018-11-19 15:45:00',
               '2018-11-19 16:45:00', '2018-11-19 17:45:00',
               '2018-11-19 18:45:00', '2018-11-19 19:45:00',
               '2018-11-19 20:45:00', '2018-11-19 21:45:00',
               '2018-11-19 22:45:00', '2018-11-19 23:45:00'],
              dtype='datetime64[ns]', freq='H')

In [91]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [92]:
df.values

array([[-0.24428391,  1.04514361,  0.14756232,  0.63490179],
       [-2.10683354,  1.78451794, -0.21649052, -0.22255866],
       [ 1.05331766, -0.29353154, -0.78141536,  0.81013583],
       [-0.64237555, -0.17111157, -1.09735772,  0.75856454],
       [-0.89854026,  0.02144639, -0.12700974,  0.38202524],
       [ 0.36228775, -0.28644202,  1.17191763, -0.61160231],
       [ 0.06062339, -0.32806802, -1.23600808,  0.38038081],
       [-0.97247772,  1.2636361 ,  0.37389194, -1.69535137],
       [ 1.75782241,  0.5624845 , -0.77230955, -0.85839733],
       [-0.31347643, -0.59868147,  1.15191389, -0.46374396]])

In [93]:
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,-0.194394,0.299939,-0.138531,-0.088565
std,1.091323,0.813385,0.861003,0.823053
min,-2.106834,-0.598681,-1.236008,-1.695351
25%,-0.834499,-0.291759,-0.779139,-0.574638
50%,-0.278880,-0.074833,-0.171750,0.078911
75%,0.286872,0.924479,0.317310,0.571683
max,1.757822,1.784518,1.171918,0.810136


In [94]:
df.T

,2018-11-19 14:45:00,2018-11-19 15:45:00,2018-11-19 16:45:00,2018-11-19 17:45:00,2018-11-19 18:45:00,2018-11-19 19:45:00,2018-11-19 20:45:00,2018-11-19 21:45:00,2018-11-19 22:45:00,2018-11-19 23:45:00
A,-0.244284,-2.106834,1.053318,-0.642376,-0.898540,0.362288,0.060623,-0.972478,1.757822,-0.313476
B,1.045144,1.784518,-0.293532,-0.171112,0.021446,-0.286442,-0.328068,1.263636,0.562484,-0.598681
C,0.147562,-0.216491,-0.781415,-1.097358,-0.127010,1.171918,-1.236008,0.373892,-0.772310,1.151914
D,0.634902,-0.222559,0.810136,0.758565,0.382025,-0.611602,0.380381,-1.695351,-0.858397,-0.463744


In [95]:
df.sort_index(axis=1,ascending=False)

,D,C,B,A
2018-11-19 14:45:00,0.634902,0.147562,1.045144,-0.244284
2018-11-19 15:45:00,-0.222559,-0.216491,1.784518,-2.106834
2018-11-19 16:45:00,0.810136,-0.781415,-0.293532,1.053318
2018-11-19 17:45:00,0.758565,-1.097358,-0.171112,-0.642376
2018-11-19 18:45:00,0.382025,-0.127010,0.021446,-0.898540
2018-11-19 19:45:00,-0.611602,1.171918,-0.286442,0.362288
2018-11-19 20:45:00,0.380381,-1.236008,-0.328068,0.060623
2018-11-19 21:45:00,-1.695351,0.373892,1.263636,-0.972478
2018-11-19 22:45:00,-0.858397,-0.772310,0.562484,1.757822
2018-11-19 23:45:00,-0.463744,1.151914,-0.598681,-0.313476


In [96]:
df.sort_values(by="C")

,A,B,C,D
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,0.380381
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,0.758565
2018-11-19 16:45:00,1.053318,-0.293532,-0.781415,0.810136
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,-0.858397
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,-0.222559
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,0.382025
2018-11-19 14:45:00,-0.244284,1.045144,0.147562,0.634902
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,-1.695351
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,-0.463744
2018-11-19 19:45:00,0.362288,-0.286442,1.171918,-0.611602


## Selection

### Getting slices

The following show how to get part of the DataFrame (i.e. not just the elements)

In [97]:
## standard and safe
print (df['A'],'\n')

## equivalent but dangerous (imagine blank spaces in the name of the column..)
print (df.A)

2018-11-19 14:45:00   -0.244284
2018-11-19 15:45:00   -2.106834
2018-11-19 16:45:00    1.053318
2018-11-19 17:45:00   -0.642376
2018-11-19 18:45:00   -0.898540
2018-11-19 19:45:00    0.362288
2018-11-19 20:45:00    0.060623
2018-11-19 21:45:00   -0.972478
2018-11-19 22:45:00    1.757822
2018-11-19 23:45:00   -0.313476
Freq: H, Name: A, dtype: float64 

2018-11-19 14:45:00   -0.244284
2018-11-19 15:45:00   -2.106834
2018-11-19 16:45:00    1.053318
2018-11-19 17:45:00   -0.642376
2018-11-19 18:45:00   -0.898540
2018-11-19 19:45:00    0.362288
2018-11-19 20:45:00    0.060623
2018-11-19 21:45:00   -0.972478
2018-11-19 22:45:00    1.757822
2018-11-19 23:45:00   -0.313476
Freq: H, Name: A, dtype: float64


In [98]:
# selecting rows by counting
print (df[0:3])

# or by index
print (df["2018-11-19 14:45:00":"2018-11-19 16:45:00"])

                            A         B         C         D
2018-11-19 14:45:00 -0.244284  1.045144  0.147562  0.634902
2018-11-19 15:45:00 -2.106834  1.784518 -0.216491 -0.222559
2018-11-19 16:45:00  1.053318 -0.293532 -0.781415  0.810136
                            A         B         C         D
2018-11-19 14:45:00 -0.244284  1.045144  0.147562  0.634902
2018-11-19 15:45:00 -2.106834  1.784518 -0.216491 -0.222559
2018-11-19 16:45:00  1.053318 -0.293532 -0.781415  0.810136


### Selection by label

In [99]:
# getting a cross section (part of the DataFrame) using a label
df.loc[dates[0]]

A   -0.244284
B    1.045144
C    0.147562
D    0.634902
Name: 2018-11-19 14:45:00, dtype: float64

In [100]:
# selecting on a multi-axis by label:
df.loc[:,['A','B']]

,A,B
2018-11-19 14:45:00,-0.244284,1.045144
2018-11-19 15:45:00,-2.106834,1.784518
2018-11-19 16:45:00,1.053318,-0.293532
2018-11-19 17:45:00,-0.642376,-0.171112
2018-11-19 18:45:00,-0.898540,0.021446
2018-11-19 19:45:00,0.362288,-0.286442
2018-11-19 20:45:00,0.060623,-0.328068
2018-11-19 21:45:00,-0.972478,1.263636
2018-11-19 22:45:00,1.757822,0.562484
2018-11-19 23:45:00,-0.313476,-0.598681


In [101]:
# showing label slicing, both endpoints are included:
df.loc['2018-11-19 18:45:00':'2018-11-19 20:45:00',['A','B']]

,A,B
2018-11-19 18:45:00,-0.898540,0.021446
2018-11-19 19:45:00,0.362288,-0.286442
2018-11-19 20:45:00,0.060623,-0.328068


In [102]:
# getting an individual element
print (df.loc[dates[1],'A'])

# equivalently
print (df.at[dates[1],'A'])

-2.1068335392488926
-2.1068335392488926


### Selecting by position

In [103]:
# select via the position of the passed integers:
print (df.iloc[3],'\n')

# notation similar to numpy/python
print (df.iloc[3:5,0:2])

A   -0.642376
B   -0.171112
C   -1.097358
D    0.758565
Name: 2018-11-19 17:45:00, dtype: float64 

                            A         B
2018-11-19 17:45:00 -0.642376 -0.171112
2018-11-19 18:45:00 -0.898540  0.021446


In [104]:
# selecting raws 1,2 and 4 for columns 0 and 2
df.iloc[[1,2,4],[0,2]]

,A,C
2018-11-19 15:45:00,-2.106834,-0.216491
2018-11-19 16:45:00,1.053318,-0.781415
2018-11-19 18:45:00,-0.898540,-0.127010


In [105]:
# slicing rows explicitly
print (df.iloc[1:3,:],'\n')

# slicing columns explicitly
print (df.iloc[:,1:3])


                            A         B         C         D
2018-11-19 15:45:00 -2.106834  1.784518 -0.216491 -0.222559
2018-11-19 16:45:00  1.053318 -0.293532 -0.781415  0.810136 

                            B         C
2018-11-19 14:45:00  1.045144  0.147562
2018-11-19 15:45:00  1.784518 -0.216491
2018-11-19 16:45:00 -0.293532 -0.781415
2018-11-19 17:45:00 -0.171112 -1.097358
2018-11-19 18:45:00  0.021446 -0.127010
2018-11-19 19:45:00 -0.286442  1.171918
2018-11-19 20:45:00 -0.328068 -1.236008
2018-11-19 21:45:00  1.263636  0.373892
2018-11-19 22:45:00  0.562484 -0.772310
2018-11-19 23:45:00 -0.598681  1.151914


In [106]:
# selecting an individual element by position
df.iloc[1,1]
df.iat[1,1]


1.7845179380196858

### Boolean index

Very powerful way of filtering out data with certain features. Notation is very similar to numpy arrays.

In [107]:
# Filter by a boolean condition on the values of a single column
df[df['B'] > 0]

,A,B,C,D
2018-11-19 14:45:00,-0.244284,1.045144,0.147562,0.634902
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,-0.222559
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,0.382025
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,-1.695351
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,-0.858397


In [108]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
df[df>0]

# a DataFrame with the same shape is returned, with NaN's where condition is not met

,A,B,C,D
2018-11-19 14:45:00,NaN,1.045144,0.147562,0.634902
2018-11-19 15:45:00,NaN,1.784518,NaN,NaN
2018-11-19 16:45:00,1.053318,NaN,NaN,0.810136
2018-11-19 17:45:00,NaN,NaN,NaN,0.758565
2018-11-19 18:45:00,NaN,0.021446,NaN,0.382025
2018-11-19 19:45:00,0.362288,NaN,1.171918,NaN
2018-11-19 20:45:00,0.060623,NaN,NaN,0.380381
2018-11-19 21:45:00,NaN,1.263636,0.373892,NaN
2018-11-19 22:45:00,1.757822,0.562484,NaN,NaN
2018-11-19 23:45:00,NaN,NaN,1.151914,NaN


### Setting

Combination of selection and setting of values

In [109]:
# setting values by label (same as by position)
df.at[dates[0],'A'] = 0

# setting and assigning a numpy array
df.loc[:,'D'] = np.array([5] * len(df))

# defining a brend new column
df['E'] = np.arange(len(df))*0.5

# defining a brend new column by means of a pd.Series: indexes must be the same!
df['E prime'] = pd.Series(np.arange(len(df))*2, index=df.index)


In [110]:
def dcos(theta):
    theta = theta*(np.pi/180)
    return np.cos(theta)
 
df['cosine'] = pd.Series(df["E"].apply(dcos), index=df.index)
df

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,1.045144,0.147562,5,0.0,0,1.000000
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,5,0.5,2,0.999962
2018-11-19 16:45:00,1.053318,-0.293532,-0.781415,5,1.0,4,0.999848
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,5,1.5,6,0.999657
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,5,2.0,8,0.999391
2018-11-19 19:45:00,0.362288,-0.286442,1.171918,5,2.5,10,0.999048
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,5,3.0,12,0.998630
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,5,3.5,14,0.998135
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,5,4.0,16,0.997564
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,5,4.5,18,0.996917


In [111]:
# another example of global setting
df2=df.copy()
df2[df2>0] = -df2
df2

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,-1.045144,-0.147562,-5,0.0,0,-1.000000
2018-11-19 15:45:00,-2.106834,-1.784518,-0.216491,-5,-0.5,-2,-0.999962
2018-11-19 16:45:00,-1.053318,-0.293532,-0.781415,-5,-1.0,-4,-0.999848
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,-5,-1.5,-6,-0.999657
2018-11-19 18:45:00,-0.898540,-0.021446,-0.127010,-5,-2.0,-8,-0.999391
2018-11-19 19:45:00,-0.362288,-0.286442,-1.171918,-5,-2.5,-10,-0.999048
2018-11-19 20:45:00,-0.060623,-0.328068,-1.236008,-5,-3.0,-12,-0.998630
2018-11-19 21:45:00,-0.972478,-1.263636,-0.373892,-5,-3.5,-14,-0.998135
2018-11-19 22:45:00,-1.757822,-0.562484,-0.772310,-5,-4.0,-16,-0.997564
2018-11-19 23:45:00,-0.313476,-0.598681,-1.151914,-5,-4.5,-18,-0.996917


### Dropping

N.B.: dropping doesn't act permanently on the DataFrame, i.e. to get that do :
```python
df = df.drop(....)
```

In [112]:
# Dropping by column
df.drop(['E prime'], axis=1)

#which is equivalent to
df.drop(columns=['E prime'])

,A,B,C,D,E,cosine
2018-11-19 14:45:00,0.000000,1.045144,0.147562,5,0.0,1.000000
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,5,0.5,0.999962
2018-11-19 16:45:00,1.053318,-0.293532,-0.781415,5,1.0,0.999848
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,5,1.5,0.999657
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,5,2.0,0.999391
2018-11-19 19:45:00,0.362288,-0.286442,1.171918,5,2.5,0.999048
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,5,3.0,0.998630
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,5,3.5,0.998135
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,5,4.0,0.997564
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,5,4.5,0.996917


In [113]:
# Dropping by raws
# save and always working
df.drop(df.index[[1,2,3,4]])

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,1.045144,0.147562,5,0.0,0,1.000000
2018-11-19 19:45:00,0.362288,-0.286442,1.171918,5,2.5,10,0.999048
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,5,3.0,12,0.998630
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,5,3.5,14,0.998135
2018-11-19 22:45:00,1.757822,0.562484,-0.772310,5,4.0,16,0.997564
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,5,4.5,18,0.996917


In [114]:
# something like df.drop('index_name') 
# would work but the type of index must be specificed, 
# in particular with DatetimeIndex
df.drop(pd.to_datetime("2018-11-19 22:45:00"))

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,1.045144,0.147562,5,0.0,0,1.000000
2018-11-19 15:45:00,-2.106834,1.784518,-0.216491,5,0.5,2,0.999962
2018-11-19 16:45:00,1.053318,-0.293532,-0.781415,5,1.0,4,0.999848
2018-11-19 17:45:00,-0.642376,-0.171112,-1.097358,5,1.5,6,0.999657
2018-11-19 18:45:00,-0.898540,0.021446,-0.127010,5,2.0,8,0.999391
2018-11-19 19:45:00,0.362288,-0.286442,1.171918,5,2.5,10,0.999048
2018-11-19 20:45:00,0.060623,-0.328068,-1.236008,5,3.0,12,0.998630
2018-11-19 21:45:00,-0.972478,1.263636,0.373892,5,3.5,14,0.998135
2018-11-19 23:45:00,-0.313476,-0.598681,1.151914,5,4.5,18,0.996917


## Missing data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations.

In [115]:
df_wNan = df[df>0]
df_wNan

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,NaN,1.045144,0.147562,5,NaN,NaN,1.000000
2018-11-19 15:45:00,NaN,1.784518,NaN,5,0.5,2.0,0.999962
2018-11-19 16:45:00,1.053318,NaN,NaN,5,1.0,4.0,0.999848
2018-11-19 17:45:00,NaN,NaN,NaN,5,1.5,6.0,0.999657
2018-11-19 18:45:00,NaN,0.021446,NaN,5,2.0,8.0,0.999391
2018-11-19 19:45:00,0.362288,NaN,1.171918,5,2.5,10.0,0.999048
2018-11-19 20:45:00,0.060623,NaN,NaN,5,3.0,12.0,0.998630
2018-11-19 21:45:00,NaN,1.263636,0.373892,5,3.5,14.0,0.998135
2018-11-19 22:45:00,1.757822,0.562484,NaN,5,4.0,16.0,0.997564
2018-11-19 23:45:00,NaN,NaN,1.151914,5,4.5,18.0,0.996917


In [116]:
# dropping raws with at least a Nan
df_wNan.dropna(how='any')

,A,B,C,D,E,E prime,cosine


In [117]:
# getting a mask
df_wNan.isna()
#df_wNan.notna()

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,True,False,False,False,True,True,False
2018-11-19 15:45:00,True,False,True,False,False,False,False
2018-11-19 16:45:00,False,True,True,False,False,False,False
2018-11-19 17:45:00,True,True,True,False,False,False,False
2018-11-19 18:45:00,True,False,True,False,False,False,False
2018-11-19 19:45:00,False,True,False,False,False,False,False
2018-11-19 20:45:00,False,True,True,False,False,False,False
2018-11-19 21:45:00,True,False,False,False,False,False,False
2018-11-19 22:45:00,False,False,True,False,False,False,False
2018-11-19 23:45:00,True,True,False,False,False,False,False


In [118]:
# filling missing data
df_wNan.fillna(value=0)

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,1.045144,0.147562,5,0.0,0.0,1.000000
2018-11-19 15:45:00,0.000000,1.784518,0.000000,5,0.5,2.0,0.999962
2018-11-19 16:45:00,1.053318,0.000000,0.000000,5,1.0,4.0,0.999848
2018-11-19 17:45:00,0.000000,0.000000,0.000000,5,1.5,6.0,0.999657
2018-11-19 18:45:00,0.000000,0.021446,0.000000,5,2.0,8.0,0.999391
2018-11-19 19:45:00,0.362288,0.000000,1.171918,5,2.5,10.0,0.999048
2018-11-19 20:45:00,0.060623,0.000000,0.000000,5,3.0,12.0,0.998630
2018-11-19 21:45:00,0.000000,1.263636,0.373892,5,3.5,14.0,0.998135
2018-11-19 22:45:00,1.757822,0.562484,0.000000,5,4.0,16.0,0.997564
2018-11-19 23:45:00,0.000000,0.000000,1.151914,5,4.5,18.0,0.996917


Fill gaps forward or backward by propagating non-NA values forward or backward:

In [119]:
df_wNan.fillna(method='pad')

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,NaN,1.045144,0.147562,5,NaN,NaN,1.000000
2018-11-19 15:45:00,NaN,1.784518,0.147562,5,0.5,2.0,0.999962
2018-11-19 16:45:00,1.053318,1.784518,0.147562,5,1.0,4.0,0.999848
2018-11-19 17:45:00,1.053318,1.784518,0.147562,5,1.5,6.0,0.999657
2018-11-19 18:45:00,1.053318,0.021446,0.147562,5,2.0,8.0,0.999391
2018-11-19 19:45:00,0.362288,0.021446,1.171918,5,2.5,10.0,0.999048
2018-11-19 20:45:00,0.060623,0.021446,1.171918,5,3.0,12.0,0.998630
2018-11-19 21:45:00,0.060623,1.263636,0.373892,5,3.5,14.0,0.998135
2018-11-19 22:45:00,1.757822,0.562484,0.373892,5,4.0,16.0,0.997564
2018-11-19 23:45:00,1.757822,0.562484,1.151914,5,4.5,18.0,0.996917


## Operations

Here comes the most relevant advantage of DataFrame. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a raw

In [120]:
# Some statistics (mean() just as an example)
# raws
print (df.mean(axis=0),'\n')
# columns
print (df.mean(axis=1),'\n')

A         -0.169965
B          0.299939
C         -0.138531
D          5.000000
E          2.250000
E prime    9.000000
cosine     0.998915
dtype: float64 

2018-11-19 14:45:00    1.027529
2018-11-19 15:45:00    1.137308
2018-11-19 16:45:00    1.568317
2018-11-19 17:45:00    1.655545
2018-11-19 18:45:00    2.142184
2018-11-19 19:45:00    2.820973
2018-11-19 20:45:00    2.785025
2018-11-19 21:45:00    3.451884
2018-11-19 22:45:00    3.935080
2018-11-19 23:45:00    4.105239
Freq: H, dtype: float64 



In [121]:
# global operations on columns
df.apply(np.cumsum)

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,1.045144,0.147562,5,0.0,0,1.000000
2018-11-19 15:45:00,-2.106834,2.829662,-0.068928,10,0.5,2,1.999962
2018-11-19 16:45:00,-1.053516,2.536130,-0.850344,15,1.5,6,2.999810
2018-11-19 17:45:00,-1.695891,2.365018,-1.947701,20,3.0,12,3.999467
2018-11-19 18:45:00,-2.594432,2.386465,-2.074711,25,5.0,20,4.998858
2018-11-19 19:45:00,-2.232144,2.100023,-0.902793,30,7.5,30,5.997906
2018-11-19 20:45:00,-2.171521,1.771955,-2.138801,35,10.5,42,6.996536
2018-11-19 21:45:00,-3.143998,3.035591,-1.764910,40,14.0,56,7.994670
2018-11-19 22:45:00,-1.386176,3.598075,-2.537219,45,18.0,72,8.992234
2018-11-19 23:45:00,-1.699652,2.999394,-1.385305,50,22.5,90,9.989152


In [122]:
df.apply(lambda x: x.max() - x.min())

A           3.864656
B           2.383199
C           2.407926
D           0.000000
E           4.500000
E prime    18.000000
cosine      0.003083
dtype: float64

In [123]:
# syntax is as usual similar to that of numpy arrays
df['A']+df['B']

2018-11-19 14:45:00    1.045144
2018-11-19 15:45:00   -0.322316
2018-11-19 16:45:00    0.759786
2018-11-19 17:45:00   -0.813487
2018-11-19 18:45:00   -0.877094
2018-11-19 19:45:00    0.075846
2018-11-19 20:45:00   -0.267445
2018-11-19 21:45:00    0.291158
2018-11-19 22:45:00    2.320307
2018-11-19 23:45:00   -0.912158
Freq: H, dtype: float64

Let's play it hard and load (in memory) a (relatively) large dataset

In [124]:
file_name="/Users/mzanetti/data/LEMMA2018/DT/raw/data_000636.txt"
data=pd.read_csv(file_name)
data

FileNotFoundError: File b'/Users/mzanetti/data/LEMMA2018/DT/raw/data_000636.txt' does not exist

Let's know do some operations among (elements of) columns

In [125]:
# the one-liner killing it all
data['timens']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25

NameError: name 'data' is not defined

In [126]:
# the old slooow way
def conversion(data):
    result=[]
    for i in range(len(data)): 
        result.append(data.loc[data.index[i],'TDC_MEAS']*25/30.+data.loc[data.index[i],'BX_COUNTER']*25)
    return result

data['timens']=conversion(data)

NameError: name 'data' is not defined

## Merge

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

### Concat

concatenation (adding raws) is straightforward


In [127]:
rdf = pd.DataFrame(np.random.randn(10, 4))
rdf

,0,1,2,3
0,1.215632,-0.311167,0.961634,-0.771851
1,2.028303,-0.956520,-3.029028,1.113698
2,-1.725523,1.501157,-0.303692,-0.842366
3,0.913926,-0.832713,1.822840,-0.153302
4,0.838662,-0.179755,-0.320753,0.546923
5,-0.202304,-0.541913,1.134055,-0.574369
6,-0.956944,0.672852,-1.406798,-0.848748
7,-0.535003,-1.133910,-0.990163,0.029709
8,1.929039,-0.112478,1.926560,-1.588224
9,0.896791,-0.141782,-0.197227,-0.388199


In [128]:
# divide it into pieaces raw-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
pieces

[          0         1         2         3
 0  1.215632 -0.311167  0.961634 -0.771851
 1  2.028303 -0.956520 -3.029028  1.113698
 2 -1.725523  1.501157 -0.303692 -0.842366,
           0         1         2         3
 3  0.913926 -0.832713  1.822840 -0.153302
 4  0.838662 -0.179755 -0.320753  0.546923
 5 -0.202304 -0.541913  1.134055 -0.574369
 6 -0.956944  0.672852 -1.406798 -0.848748,
           0         1         2         3
 7 -0.535003 -1.133910 -0.990163  0.029709
 8  1.929039 -0.112478  1.926560 -1.588224
 9  0.896791 -0.141782 -0.197227 -0.388199]

In [129]:
# put it back together
pd.concat(pieces)

# indexes can be ignored
#pd.concat(pieces, ignore_index=True)

# in case of dimension mismatch, Nan are added where needed

,0,1,2,3
0,1.215632,-0.311167,0.961634,-0.771851
1,2.028303,-0.956520,-3.029028,1.113698
2,-1.725523,1.501157,-0.303692,-0.842366
3,0.913926,-0.832713,1.822840,-0.153302
4,0.838662,-0.179755,-0.320753,0.546923
5,-0.202304,-0.541913,1.134055,-0.574369
6,-0.956944,0.672852,-1.406798,-0.848748
7,-0.535003,-1.133910,-0.990163,0.029709
8,1.929039,-0.112478,1.926560,-1.588224
9,0.896791,-0.141782,-0.197227,-0.388199


In [130]:
# appending a single raw (as a Series)
s = rdf.iloc[3]
rdf.append(s, ignore_index=True)


,0,1,2,3
0,1.215632,-0.311167,0.961634,-0.771851
1,2.028303,-0.956520,-3.029028,1.113698
2,-1.725523,1.501157,-0.303692,-0.842366
3,0.913926,-0.832713,1.822840,-0.153302
4,0.838662,-0.179755,-0.320753,0.546923
5,-0.202304,-0.541913,1.134055,-0.574369
6,-0.956944,0.672852,-1.406798,-0.848748
7,-0.535003,-1.133910,-0.990163,0.029709
8,1.929039,-0.112478,1.926560,-1.588224
9,0.896791,-0.141782,-0.197227,-0.388199


### Merge/Join

SQL like operations on table can be performed on DataFrames. This is all rather sophisticated, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples

In [131]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left,right,on="key")

,key,lval,rval
0,foo,1,4
1,bar,2,5


## Grouping

By “group by” we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [132]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})
gdf

,A,B,C,D
0,foo,one,1.616407,1.900573
1,bar,one,-0.627112,-0.393548
2,foo,two,-1.204514,-1.202024
3,bar,three,-1.203790,-0.150183
4,foo,two,1.710510,1.264098
5,bar,two,-0.024587,-1.903858
6,foo,one,-0.063155,0.926733
7,foo,three,0.248929,1.972903


In [133]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where number are there).
gdf.groupby('A').sum()

,C,D
A,,
bar,-1.855488,-2.447589
foo,2.308177,4.862283


## Multi-indexing


Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In essence, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1d) and DataFrame (2d).

In [134]:
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print (index,'\n')

s = pd.Series(np.random.randn(8), index=multi_index)
print (s)


NameError: name 'index' is not defined

In [135]:
# it enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A','B']).sum()

C         D
A   B                        
bar one   -0.627112 -0.393548
    three -1.203790 -0.150183
    two   -0.024587 -1.903858
foo one    1.553252  2.827307
    three  0.248929  1.972903
    two    0.505996  0.062073

In [136]:
# stack() method “compresses” a level in the DataFrame’s columns
gdf.groupby(['A','B']).sum().stack()

A    B       
bar  one    C   -0.627112
            D   -0.393548
     three  C   -1.203790
            D   -0.150183
     two    C   -0.024587
            D   -1.903858
foo  one    C    1.553252
            D    2.827307
     three  C    0.248929
            D    1.972903
     two    C    0.505996
            D    0.062073
dtype: float64

## Plotting

Just a preview, more on the next lab class!

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts.cumsum().plot()

In [ ]:
df=pd.DataFrame(np.random.randn(1000, 4), index=ts.index,columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best')